In [4]:
import tensorflow as tf
import PIL.Image as Image
import numpy as np

In [2]:
interpreter = tf.lite.Interpreter(model_path='../models/mvnet_converted.tflite')

In [3]:
print("== Input details ==")
print("name:", interpreter.get_input_details()[0]['name'])
print("shape:", interpreter.get_input_details()[0]['shape'])
print("type:", interpreter.get_input_details()[0]['dtype'])

print("\n== Output details ==")
print("name:", interpreter.get_output_details()[0]['name'])
print("shape:", interpreter.get_output_details()[0]['shape'])
print("type:", interpreter.get_output_details()[0]['dtype'])

print("\nDUMP INPUT")
print(interpreter.get_input_details()[0])
print("\nDUMP OUTPUT")
print(interpreter.get_output_details()[0])

== Input details ==
name: keras_layer_input
shape: [  1 224 224   3]
type: <class 'numpy.float32'>

== Output details ==
name: Identity
shape: [   1 1001]
type: <class 'numpy.float32'>

DUMP INPUT
{'name': 'keras_layer_input', 'index': 0, 'shape': array([  1, 224, 224,   3], dtype=int32), 'shape_signature': array([ -1, 224, 224,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}

DUMP OUTPUT
{'name': 'Identity', 'index': 176, 'shape': array([   1, 1001], dtype=int32), 'shape_signature': array([  -1, 1001], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}


In [7]:
def preprocess_inference_image(img: str):
    IMAGE_SHAPE = (224, 224)
    img = Image.open(img).resize(IMAGE_SHAPE)
    img = np.array(img)/255.0
    #img = preprocess_inference_image('../sample_image.png')
    print('Old shape', img.shape)
    img_expanded = img[np.newaxis, ...]
    print('New shape', img_expanded.shape)
    return img_expanded

In [8]:
example_img_for_tflite = preprocess_inference_image('../image02.png')

Old shape (224, 224, 3)
New shape (1, 224, 224, 3)


In [13]:
interpreter.allocate_tensors()
print("Input data shape:", example_img_for_tflite.shape)
example_img_for_tflite = np.array(example_img_for_tflite, dtype=np.float32)
print("Input data type:", example_img_for_tflite.dtype)

input_details = interpreter.get_input_details()
interpreter.set_tensor(input_details[0]['index'], example_img_for_tflite)

interpreter.invoke()

labels_path = '../ImageNetLabels.txt'
imagenet_labels = np.array(open(labels_path).read().splitlines())

output_details = interpreter.get_output_details()
output_data = interpreter.get_tensor(output_details[0]['index'])
print("\n\nPrediction results:", output_data)
print("Predicted value:", np.argmax(output_data))
print("Predicted label:", imagenet_labels[np.argmax(output_data)])



Input data shape: (1, 224, 224, 3)
Input data type: float32


Prediction results: [[ 0.15102446  0.13678665  0.5798025  ...  0.13791749 -0.2675998
   1.5221497 ]]
Predicted value: 208
Predicted label: golden retriever
